In [1]:
# @zhangjoy
# @构建主题因子
# @20180327
import os
import csv
import tushare as ts
import pandas as pd
import numpy as np
import jieba
import sys
import time
import re
from sklearn.feature_extraction.text import CountVectorizer # 词频统计
from sklearn.decomposition import LatentDirichletAllocation # LDA
from sklearn.cluster import KMeans # k均值聚类

In [2]:
comment = []
path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/cjpl/"   # 评论内容的存储路径
filenames = os.listdir(path)
filenames.sort() # 列表排序
for filename in filenames:
    filepath = path + filename
    # print(filepath)
    with open(filepath, 'r') as f:
        reader = csv.reader(f)
        comment_text = [row[7] for row in reader]
        comment.append(comment_text)
print(len(comment)) # 18

18


In [10]:
# 得到所有股票的代码和中文名字
stock_name = ts.get_stock_basics()['name'] # 股票名称
stock_code = stock_name.index.tolist() # 股票代码
# 获得所有的概念列表、行业列表、地域列表、公司名称等
concept = ts.get_concept_classified()['c_name'].drop_duplicates().values.tolist() # 概念列表 163
industry = ts.get_industry_classified('sw')['c_name'].drop_duplicates().values.tolist() # 申万行业列表 129 > 110
place = ts.get_area_classified()['area'].drop_duplicates().values.tolist() # 获取地域列表 32 省份

broker = ts.broker_tops()["broker"] # 获取营业部信息：1062
pattern = re.compile(u'.*有限.*?公司') # 最小匹配[*?]到'有限 公司'
company = [" ".join(pattern.findall(s)) for s in broker] # list
company = pd.Series(company).drop_duplicates().values.tolist() # 证券公司：94

[Getting data:]###############################

In [22]:
# 将上述信息作为新词录入词典
for d in stock_name:
    jieba.add_word(d)
for d in stock_code:
    jieba.add_word(d)
for d in concept:
    jieba.add_word(d)
for d in industry:
    jieba.add_word(d)
# for d in place:
#     jieba.add_word(d)    
for d in company:
    jieba.add_word(d)

In [23]:
# 启用自定义词典
jieba.load_userdict("/Users/zhangjoy/Documents/vscode/py_project/lda/data/userdict.txt")

In [24]:
# 停用词列表
stop =[line.strip() for line in open('/Users/zhangjoy/Documents/vscode/py_project/lda/data/stopwords.txt').readlines()]

In [25]:
pattern = re.compile(u'[\u4e00-\u9fa5]{2,20}') # 中文

In [26]:
texts = [] # 清洗后文本
showi = 0
showj = 1
# print(comment[showi][showj])
for i in range(len(comment)):
    text = []
    t1 = time.time()
    for sentence in comment[i]:
        chinese = pattern.findall(sentence) # 匹配中文：先分句
        seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
        c = []
        for ss in seg:
            for s in ss:
                if s not in stop: # 过滤停用词
                    c.append(s)
        text.append(" ".join(c))
    t2 = time.time()
    texts.append(text)
    print("comment_" + str(i) + " preprocessing time using %.5f s" %(t2-t1))
# 17:41-18:20 约 40 m

comment_0 preprocessing time using 42.37331 s
comment_1 preprocessing time using 86.78557 s
comment_2 preprocessing time using 91.81986 s
comment_3 preprocessing time using 138.17313 s
comment_4 preprocessing time using 156.55418 s
comment_5 preprocessing time using 137.22266 s
comment_6 preprocessing time using 120.33943 s
comment_7 preprocessing time using 113.42303 s
comment_8 preprocessing time using 96.95149 s
comment_9 preprocessing time using 81.75059 s
comment_10 preprocessing time using 84.84410 s
comment_11 preprocessing time using 106.20842 s
comment_12 preprocessing time using 100.54806 s
comment_13 preprocessing time using 112.86744 s
comment_14 preprocessing time using 105.31068 s
comment_15 preprocessing time using 140.74239 s
comment_16 preprocessing time using 192.13992 s
comment_17 preprocessing time using 342.51002 s
股市 收盘 红四月 爽约 大盘 震荡 上行 判若两人 一路 阴跌 上证综指 创业板指 均 小幅 下跌 收场 债市 风险 暴露 一时间 风声鹤唳 机构 惊慌失色 前 股市 债市 表现 动荡 公募基金 表现 分化 很大 但前 里 基金 取得 不错 正收 基金君 一起 主动 股票型基金 平均 跌 次新基金 逆

In [47]:
# 计算个股主题因子得分 预备工作
# 股票池：沪深300
hs300 = ts.get_hs300s() 
hs300_code = hs300['code'].values.tolist()
hs300_name = hs300['name'].values.tolist()
concept_df = ts.get_concept_classified()
industry_df = ts.get_industry_classified('sw')

# 筛选出与股票池相关的概念
concept = concept_df.loc[concept_df['code'].isin(hs300_code)]['c_name'].drop_duplicates().values.tolist() # 146
# 筛选出与股票池相关的行业
industry = industry_df.loc[industry_df['code'].isin(hs300_code)]['c_name'].drop_duplicates().values.tolist() # 104

# 统计词频需要的模块
from collections import Counter

http://www.csindex.com.cn/uploads/file/autofile/closeweight/000300closeweight.xls


In [62]:

# 1、全部文档前几个主题，假设1个文档可以属于多个主题，首先一定属于1个主题，后面几个主题只要满足在1.5倍的差距内，即 0.4*1.5 < 0.6 0.3*1.5 0.5
def print_top_topics_number(model, count_matrix, k, topic_names, toptopicnames_list):
    doctopic_matrix = model.fit_transform(count_matrix)
    for doc_idx, doc in enumerate(doctopic_matrix):
        for i in doc.argsort()[:-k - 1:-1]:
            if i == doc.argsort()[:-k - 1:-1][0]:
                toptopicnames_list.append(topic_names[i])
                continue
            elif round(doc[i], 2)*1.5 >= round(doc[doc.argsort()[:-k - 1:-1][0]], 2):
                toptopicnames_list.append(topic_names[i])
                
# 3、热门主题关键词筛选，为每个主题选择前 N 个主题词
def print_hottopic_keywords(model, topicnames, word_names, topickeywords_name, topickeywords_prob, n_top_words):
    topicword_matrix = model.components_[topicnames]
    for topic_idx,topic in enumerate(topicword_matrix):
        topickeyword_name = [word_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topickeywords_name.append(topickeyword_name)
        topickeyword_prob = [round(topic[i], 2) for i in topic.argsort()[:-n_top_words - 1:-1]]
        topickeywords_prob.append(topickeyword_prob)
        
# 4、计算每个主题：主题关键词概念/行业 归类信息
def concept_industry_info(topickeywords_name, topickeywords_prob, concept_stay_list, concept_prob_list, industry_stay_list, industry_prob_list, topickeywords_weight_list):
    for i in range(len(topickeywords_name)):
        concept_num, industry_num = 0, 0
        concept_stay, concept_prob = [], []
        industry_stay, industry_prob = [], []
        for j in range(topN):
            name = topickeywords_name[i][j]
            prob = topickeywords_prob[i][j]
            for con in concept: # 与关键词相关概念 
                if name in con:
                    concept_stay.append(con)
                    concept_prob.append(prob)
                    concept_num = concept_num + 1
            for ind in industry: # 与关键词相关行业 
                if name in ind:
                    industry_stay.append(ind)
                    industry_prob.append(prob)
                    industry_num = industry_num + 1
        # 主题类别权重
        if concept_num > 0 and industry_num > 0: # and place_num > 0:
            topickeywords_weight = [0.5, 0.5]
        elif concept_num == 0:
            topickeywords_weight = [0, 1]
        else:
            topickeywords_weight = [1, 0]
        
        concept_stay_list.append(concept_stay)
        concept_prob_list.append(concept_prob)
        industry_stay_list.append(industry_stay)
        industry_prob_list.append(industry_prob)
        topickeywords_weight_list.append(topickeywords_weight)

# 辅助函数：两列表对应元素相乘并求和
def list_sum(a, b): 
    s = sum([a[i]*b[i] for i in range(len(a))])
    return s

# 5、匹配个股信息，计算因子得分，并标准化
# # 获取单只股票的概念信息、行业信息 --> 从而得到股票主题得分
def stock(stock_topicscore_list,concept_stay_list,concept_prob_list,industry_stay_list,industry_prob_list,topickeywords_weight_list):
    for stock_code in hs300_code:
        stock_concept = concept_df.loc[concept_df['code'] == stock_code]['c_name'].tolist()
        stock_industry = industry_df.loc[industry_df['code'] == stock_code]['c_name'].tolist()
        stock_topicscore = []
        for i in range(topk): # 先计算每个主题得分，再加权得到总主题得分
            concept_stay = concept_stay_list[i]
            concept_prob = concept_prob_list[i]
            industry_stay = industry_stay_list[i]
            industry_prob = industry_prob_list[i]
            topickeywords_weight = topickeywords_weight_list[i]
            
            concept_score = sum([concept_prob[i] for i in range(len(concept_stay)) if concept_stay[i] in stock_concept])
            industry_score = sum([industry_prob[i] for i in range(len(industry_stay)) if industry_stay[i] in stock_industry])
            ciscore_list = [concept_score,industry_score]
            topicscore = list_sum(ciscore_list, topickeywords_weight)
            stock_topicscore.append(topicscore)

        stock_topicscore = list_sum(stock_topicscore, top_topicweight)
        stock_topicscore_list.append(stock_topicscore)


In [129]:
# 主函数
def main(corpus,topic_factor):
    # LDA主题建模
    # 构建 Document-word matrix
    count_vectorizer = CountVectorizer(stop_words=stop, min_df=0.005) # 特征选择：必须在文档总数的 0.5% 中出现过
    count_matrix = count_vectorizer.fit_transform(corpus) # 词频矩阵
    wordnames_list = count_vectorizer.get_feature_names() # 单词(特征)名称
    print(count_matrix.shape) # ok
    # 训练模型
    k = 20 # 选择主题数
    topicnames_list = [i for i in range(k)] # 主题名称
    t1 = time.time()
    lda = LatentDirichletAllocation(n_components=k, max_iter=50,learning_method='batch',random_state=1234)
    lda.fit(count_matrix)
    t2 = time.time()
    print("lda modeling time using %.5f s" %(t2-t1)) # ok 1 m
    # 结果输出
    t3 = time.time()
    toptopicnames_list = [] # 主题名称，选择 topk = 5
    print_top_topics_number(lda, count_matrix, k, topicnames_list, toptopicnames_list)
    t4 = time.time()
    print("calculate top topicname per doc using %.5f s" %(t4-t3)) # ok 1 m
    # 2.选择热门的前3-5个主题，要归一化
    t5 = time.time()
    topk = 5
    top_topicnames = [Counter(toptopicnames_list).most_common(topk)[i][0] for i in range(topk)]
    top_topicweight = [Counter(toptopicnames_list).most_common(topk)[i][1] for i in range(topk)]
    top_topicweight = [i/sum(top_topicweight) for i in top_topicweight] # 归一化
    t6 = time.time()
    print("statistic top topicnames using %.5f s" %(t6-t5)) # ok
    # 3.热门主题关键词筛选，为每个主题选择前 topN 个主题词
    t7 = time.time()
    topN = 100
    top_keywords_name, top_keywords_prob = [],[]
    print_hottopic_keywords(lda, top_topicnames, wordnames_list, top_keywords_name, top_keywords_prob , topN)
    t8 = time.time()
    print("choose hottopic topkeywords using %.5f s" %(t8-t7)) # ok
    # 4.为主题关键词归类，正则匹配，找出与关键词相关的概念/行业等
    t9 = time.time()
    concept_stay_list, concept_prob_list, industry_stay_list, industry_prob_list, topickeywords_weight_list = [],[],[],[],[]
    concept_industry_info(top_keywords_name, top_keywords_prob, concept_stay_list, concept_prob_list, industry_stay_list, industry_prob_list, topickeywords_weight_list)
    t10 = time.time()
    print("get concept industry info using %.5f s" %(t10-t9)) # ok
    # 5.计算个股主题因子值
    t11 = time.time()
    stock_topicscore_list = []
    stock(stock_topicscore_list,concept_stay_list,concept_prob_list,industry_stay_list,industry_prob_list,topickeywords_weight_list)
    # 因子值标准化
    stock_topicscore_list = [(i-min(stock_topicscore_list))/(max(stock_topicscore_list)-min(stock_topicscore_list)) for i in stock_topicscore_list]
    stock_topicfactor.append(stock_topicscore_list)
    t12 = time.time()
    print("calculate stock topicfactor using %.5f s" %(t12-t11)) # ok

In [82]:
corpus = texts[0]
stock_topicfactor = []
main(corpus,stock_topicfactor)

(2010, 6610)
lda modeling time using 63.03303 s
calculate top topicname per doc using 62.30640 s
statistic top topicnames using 0.00096 s
choose hottopic topkeywords using 0.00616 s
[['资金', '反弹', '震荡', '大盘', '行情', '板块', '出现', '个股', '投资者', '指数', '短期', '上涨', '下跌', '走势', '股市', '持续', '调整', '主力', '机会', '两市', '股指', '涨幅', '预期', '风险', '关注', '跌幅', '表现', '创业板', '明显', '利好', '后市', '大幅', '业绩', '空间', '当前', '成交量', '热点', '有望', '因素', '连续', '杀跌', '突破', '支撑', '操作', '依然', '均线', '概率', '趋势', '估值', '向上', '策略', '回升', '压力', '盘面', '收盘', '形成', '券商', '上行', '行业', '尾盘', '技术', '较大', '维持', '处于', '盘中', '短线', '成交', '回落', '格局', '整体', '交易日', '附近', '再度', '影响', '仓位', '弱势', '前期', '建议', '方面', '小幅', '强势', '放量', '品种', '机构', '总体', '数据', '偏好', '情绪', '萎缩', '新高', '经济', '不断', '延续', '阶段', '缩量', '谨慎', '午后', '比较', '相对', '大跌'], ['中国', '股市', '经济', '全球', '美国', '这种', '需要', '投资者', '应该', '日本', '时间', '未来', '过去', '发生', '重要', '世界', '投资', '情况', '国家', '一次', '看到', '观点', '泡沫', '影响', '这是', '知道', '出现', '股票', '趋势', '机构', '最大', '机会', '长期', '希望', '新兴',

In [115]:
topicfactor_df = pd.DataFrame(stock_topicfactor[0])
topicfactor_df.columns = ['topicfactor']
file_df = ts.get_hs300s()[['code','name']].join(topicfactor_df)
file_df.to_csv("/Users/zhangjoy/Documents/vscode/py_project/lda/stock_topicfactor_0.csv")

http://www.csindex.com.cn/uploads/file/autofile/closeweight/000300closeweight.xls


In [127]:
for i in range(3,10):
    t1 = time.time()
    corpus = texts[i]
    stock_topicfactor = []
    main(corpus,stock_topicfactor)
    t2 = time.time()
    print("texts_" + str(i) + " all time using %.5f s" %(t2-t1))
    print()
    topicfactor_df = pd.DataFrame(stock_topicfactor[0])
    factorname = 'topicfactor_' + str(i)
    topicfactor_df.columns = [factorname]
    file_df = file_df.join(topicfactor_df)

file_df.to_csv("/Users/zhangjoy/Documents/vscode/py_project/lda/data/stock_topicfactor_9.csv")


(5910, 7035)
lda modeling time using 191.75736 s
calculate top topicname per doc using 197.56386 s
statistic top topicnames using 0.00290 s
choose hottopic topkeywords using 0.00658 s
get concept industry info using 0.00406 s
calculate stock topicfactor using 0.60861 s
texts_3 all time using 392.48608 s
(7428, 6594)
lda modeling time using 215.91086 s
calculate top topicname per doc using 221.84550 s
statistic top topicnames using 0.00394 s
choose hottopic topkeywords using 0.00610 s
get concept industry info using 0.00404 s
calculate stock topicfactor using 0.60445 s
texts_4 all time using 441.19337 s
(6840, 6429)
lda modeling time using 199.36633 s
calculate top topicname per doc using 198.64415 s
statistic top topicnames using 0.00326 s
choose hottopic topkeywords using 0.00605 s
get concept industry info using 0.00406 s
calculate stock topicfactor using 0.59733 s
texts_5 all time using 401.12511 s
(5567, 6765)
lda modeling time using 164.76992 s
calculate top topicname per doc usin

In [130]:
for i in range(10,18):
    t1 = time.time()
    corpus = texts[i]
    stock_topicfactor = []
    main(corpus,stock_topicfactor)
    t2 = time.time()
    print("texts_" + str(i) + " all time using %.5f s" %(t2-t1))
    print()
    topicfactor_df = pd.DataFrame(stock_topicfactor[0])
    factorname = 'topicfactor_' + str(i)
    topicfactor_df.columns = [factorname]
    file_df = file_df.join(topicfactor_df)

file_df.to_csv("/Users/zhangjoy/Documents/vscode/py_project/lda/data/stock_topicfactor_17.csv")

(3843, 7171)
lda modeling time using 101.77709 s
calculate top topicname per doc using 103.42926 s
statistic top topicnames using 0.00253 s
choose hottopic topkeywords using 0.00852 s
get concept industry info using 0.00450 s
calculate stock topicfactor using 0.54466 s

texts_10 all time using 207.21305 s
(4664, 7198)
lda modeling time using 129.01290 s
calculate top topicname per doc using 132.29962 s
statistic top topicnames using 0.00232 s
choose hottopic topkeywords using 0.00652 s
get concept industry info using 0.00391 s
calculate stock topicfactor using 0.55418 s

texts_11 all time using 263.64932 s
(4741, 6810)
lda modeling time using 126.77885 s
calculate top topicname per doc using 139.02508 s
statistic top topicnames using 0.00237 s
choose hottopic topkeywords using 0.00639 s
get concept industry info using 0.00393 s
calculate stock topicfactor using 0.63553 s

texts_12 all time using 268.09782 s
(5181, 6757)
lda modeling time using 153.35137 s
calculate top topicname per do

In [ ]:
file_df